In [1]:
import ray
import pandas as pd
import warnings
import math
import time
import os

# Suppress noisy requests warnings.
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

* Ray es un proyecto de código abierto para Python paralelo y distribuido.

* La computación paralela y distribuida es un elemento básico de las aplicaciones modernas. Necesitamos aprovechar varios núcleos o varias máquinas para acelerar las aplicaciones o ejecutarlas a gran escala. La infraestructura para rastrear la web y responder a las consultas de búsqueda no son programas de un solo hilo que se ejecutan en el portátil de alguien, sino conjuntos de servicios que se comunican e interactúan entre sí.

### ¿Por qué Ray?

Muchos tutoriales explican cómo utilizar el módulo de multiprocesamiento de Python. Desafortunadamente, el módulo de multiprocesamiento está severamente limitado en su capacidad para manejar los requerimientos de las aplicaciones modernas. Estos requisitos incluyen los siguientes:

* Ejecutar el mismo código en más de una máquina.
* Construir microservicios y actores que tengan estado y puedan comunicarse.
* Manejar con gracia los fallos de la máquina.
* Manejo eficiente de objetos grandes y datos numéricos.

Ray aborda todos estos puntos, simplifica las cosas sencillas y hace posible un comportamiento complejo.

### Conceptos necesarios
La programación tradicional se basa en dos conceptos fundamentales: funciones y clases. Utilizando estos bloques de construcción, los lenguajes de programación nos permiten construir innumerables aplicaciones.

Sin embargo, cuando migramos nuestras aplicaciones al entorno distribuido, los conceptos suelen cambiar.

En un extremo del espectro, tenemos herramientas como OpenMPI, el multiprocesamiento de Python y ZeroMQ, que proporcionan primitivas de bajo nivel para enviar y recibir mensajes. Estas herramientas son muy potentes, pero proporcionan una abstracción diferente, por lo que las aplicaciones de un solo hilo deben reescribirse desde cero para utilizarlas.

En el otro extremo del espectro, tenemos herramientas de dominio específico como TensorFlow para la formación de modelos, Spark para el procesamiento de datos y SQL, y Flink para el procesamiento de flujos. Estas herramientas proporcionan abstracciones de alto nivel como redes neuronales, conjuntos de datos y flujos. Sin embargo, como difieren de las abstracciones utilizadas para la programación en serie, las aplicaciones deben reescribirse desde cero para aprovecharlas.

<br>

![Image](https://miro.medium.com/max/720/1*CFtsBP134JiGeQQqoKNcXw.webp)

Ray ocupa un lugar intermedio único. En lugar de introducir nuevos conceptos. Ray toma los conceptos existentes de funciones y clases y los traslada al entorno distribuido como tareas y actores. Esta elección de API permite paralelizar aplicaciones en serie sin grandes modificaciones.

### Iniciar Ray

El comando `ray.init()` inicia todos los procesos Ray relevantes. En un cluster, esta es la única línea que necesita cambiar (necesitamos pasar la dirección del cluster). Estos procesos incluyen lo siguiente:

* Un número de procesos trabajadores para ejecutar funciones Python en paralelo (aproximadamente un trabajador por núcleo de CPU).
* Un proceso planificador para asignar "tareas" a los trabajadores (y a otras máquinas). Una tarea es la unidad de trabajo programada por Ray y corresponde a una invocación de función o método.
* Un almacén de objetos en memoria compartida para compartir objetos eficientemente entre los trabajadores (sin crear copias).
* Una base de datos en memoria para almacenar los metadatos necesarios para volver a ejecutar tareas en caso de fallo de la máquina.

Los trabajadores Ray son procesos separados en lugar de hilos porque el soporte para multihilos en Python es muy limitado debido al bloqueo global del intérprete.

In [ ]:
ray.init()

### Paralelismo con tareas

Para convertir una función f de Python en una "función remota" (una función que puede ejecutarse de forma remota y asíncrona), declaramos la función con el decorador `@ray.remote`. Entonces, las invocaciones a la función a través de f.remote() devolverán inmediatamente futuros (un futuro es una referencia a la eventual salida), y la ejecución real de la función tendrá lugar en segundo plano (nos referimos a esta ejecución como una tarea).

In [ ]:
@ray.remote
def f(x):
    time.sleep(1)
    return x

In [ ]:
# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))

In [ ]:
results = ray.get(result_ids)
results

Dado que la llamada a `f.remote(i)` retorna inmediatamente, se pueden ejecutar cuatro copias de f en paralelo simplemente ejecutando esa línea cuatro veces.

### Dependencia de tareas

Las tareas también pueden depender de otras tareas. A continuación, la tarea multiply_matrices utiliza las salidas de las dos tareas create_matrix, por lo que no comenzará a ejecutarse hasta que se hayan ejecutado las dos primeras tareas. Las salidas de las dos primeras tareas se pasarán automáticamente como argumentos a la tercera tarea y los futuros se sustituirán por sus valores correspondientes). De este modo, las tareas pueden componerse con dependencias DAG arbitrarias.

In [ ]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

x_id = create_matrix.remote([1000, 1000])
y_id = create_matrix.remote([1000, 1000])
z_id = multiply_matrices.remote(x_id, y_id)

# Get the results.
z = ray.get(z_id)

In [ ]:
z

### Agregación eficaz de valores

Las relaciones de tareas pueden utilizarse de formas mucho más sofisticadas. Por ejemplo, supongamos que queremos agregar 8 valores. Este ejemplo utiliza la suma de números enteros, pero en muchas aplicaciones, la agregación de grandes vectores a través de múltiples máquinas puede ser un cuello de botella. En este caso, el cambio de una sola línea de código puede cambiar el tiempo de ejecución de la agregación de lineal a logarítmico en el número de valores que se agregan.

<br>

![Image](https://miro.medium.com/max/720/1*vHz3troEmr4uLns0V8VmdA.webp)

Como se ha descrito anteriormente, para alimentar la salida de una tarea como entrada en una tarea posterior, basta con pasar el futuro devuelto por la primera tarea como argumento en la segunda tarea. El programador de Ray tendrá en cuenta automáticamente esta dependencia entre tareas. La segunda tarea no se ejecutará hasta que la primera haya terminado, y la salida de la primera tarea se enviará automáticamente a la máquina en la que se esté ejecutando la segunda.

In [ ]:
import time

@ray.remote
def add(x, y):
    time.sleep(1)
    return x + y

# Aggregate the values slowly. This approach takes O(n) where n is the
# number of values being aggregated. In this case, 7 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(id1, 3)
id3 = add.remote(id2, 4)
id4 = add.remote(id3, 5)
id5 = add.remote(id4, 6)
id6 = add.remote(id5, 7)
id7 = add.remote(id6, 8)
result = ray.get(id7)

# Aggregate the values in a tree-structured pattern. This approach
# takes O(log(n)). In this case, 3 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(3, 4)
id3 = add.remote(5, 6)
id4 = add.remote(7, 8)
id5 = add.remote(id1, id2)
id6 = add.remote(id3, id4)
id7 = add.remote(id5, id6)
result = ray.get(id7)

In [ ]:
result

In [ ]:
# Slow approach.
values = [1, 2, 3, 4, 5, 6, 7, 8]
while len(values) > 1:
    values = [add.remote(values[0], values[1])] + values[2:]
result = ray.get(values[0])


# Fast approach.
values = [1, 2, 3, 4, 5, 6, 7, 8]
while len(values) > 1:
    values = values[2:] + [add.remote(values[0], values[1])]
result = ray.get(values[0])

### De las clases a los actores
Es un reto escribir aplicaciones interesantes sin usar clases, y esto es tan cierto en el entorno distribuido como en un único núcleo.

Ray te permite tomar una clase Python y declararla con el decorador `@ray.remote`. Cada vez que se instancia la clase, Ray crea un nuevo "actor", que es un proceso que se ejecuta en algún lugar del clúster y contiene una copia del objeto. Las invocaciones a métodos en ese actor se convierten en tareas que se ejecutan en el proceso del actor y pueden acceder y mutar el estado del actor. De este modo, los actores permiten que el estado mutable se comparta entre múltiples tareas de un modo que las funciones remotas no permiten.

Los actores individuales ejecutan métodos en serie (cada método individual es atómico) para que no haya condiciones de carrera. El paralelismo puede lograrse mediante la creación de múltiples actores.

In [ ]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.x = 0
    
    def inc(self):
        self.x += 1
    
    def get_value(self):
        return self.x

# Create an actor process.
c = Counter.remote()

# Check the actor's counter value.
print(ray.get(c.get_value.remote()))  # 0

# Increment the counter twice and check the value again.
c.inc.remote()
c.inc.remote()
print(ray.get(c.get_value.remote()))  # 2

El ejemplo anterior es el uso más simple posible de los actores. La línea `Counter.remote()` crea un nuevo proceso actor, que tiene una copia del objeto `Counter`. Las llamadas a `c.get_value.remote()` y `c.inc.remote()` ejecutan tareas en el proceso actor remoto y mutan el estado del actor.

#### Manejadores de Actor

En el ejemplo anterior, sólo invocamos métodos en el actor desde el script principal de Python. Uno de los aspectos más poderosos de los actores es que podemos pasar manejadores a un actor, lo que permite a otros actores u otras tareas invocar métodos en el mismo actor.

El siguiente ejemplo crea un actor que almacena mensajes. Varias tareas de trabajo envían mensajes al actor repetidamente, y el script principal de Python lee los mensajes periódicamente.

In [ ]:
@ray.remote
class MessageActor(object):
    def __init__(self):
        self.messages = []
    
    def add_message(self, message):
        self.messages.append(message)
    
    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages


# Define a remote function which loops around and pushes
# messages to the actor.
@ray.remote
def worker(message_actor, j):
    for i in range(100):
        time.sleep(1)
        message_actor.add_message.remote(
            "Message {} from worker {}.".format(i, j))


# Create a message actor.
message_actor = MessageActor.remote()

# Start 3 tasks that push messages to the actor.
[worker.remote(message_actor, j) for j in range(3)]

# Periodically get the messages and print them.
for _ in range(10):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New messages:", new_messages)
    time.sleep(1)

* Los actores son extremadamente potentes. Permiten tomar una clase Python e instanciarla como un microservicio que puede ser consultado desde otros actores y tareas e incluso desde otras aplicaciones.

* Las tareas y los actores son las abstracciones básicas que proporciona Ray. Estos dos conceptos son muy generales y se pueden utilizar para implementar aplicaciones sofisticadas, incluidas las bibliotecas integradas de Ray para el aprendizaje por refuerzo, el ajuste de hiperparámetros, la aceleración de Pandas, y mucho más.

In [2]:
ray.shutdown()
ray.init()

2022-12-14 14:14:01,782	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 10.3.27.163:6379...
2022-12-14 14:14:01,794	INFO worker.py:1538 -- Connected to Ray cluster.


Python version:,3.8.10
Ray version:,2.2.0


In [19]:
direct = '../PySpark/flights_data.parquet/'
list_files = os.listdir(direct)
filter_files = []

for file in list_files:
    if file[-8:] == '.parquet':
        filter_files.append(os.path.abspath(direct + file))
        
# Read Parquet files in parallel
ds = ray.data.read_parquet(filter_files)

Parquet Files Sample:  50%|███████████           | 1/2 [00:00<00:00,  3.04it/s]
2022-12-14 14:25:41,249	WARNING read_api.py:326 -- ⚠️  The number of blocks in this dataset (5) limits its parallelism to 5 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Parquet Files Sample: 100%|██████████████████████| 2/2 [00:00<00:00,  5.75it/s]


In [20]:
# Create 10 blocks for parallelism
ds = ds.repartition(10)

Repartition: 100%|██████████| 10/10 [00:00<00:00, 11.24it/s]


In [21]:
ds.limit(5).to_pandas()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408,-22,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741,-9,0,0,None,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811,5,0,0,None,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756,-9,0,0,None,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259,-21,0,0,None,NaN,NaN,NaN,NaN,NaN


In [22]:
def transform_batch(df: pd.DataFrame) -> pd.DataFrame:
    return df[(df["ARRIVAL_TIME"] < 1000) & (df["ARRIVAL_DELAY"] > 5)]

In [23]:
transformed_ds = ds.map_batches(transform_batch)

Map_Batches: 100%|██████████| 10/10 [00:01<00:00,  7.24it/s]


In [24]:
transformed_ds.limit(10).to_pandas()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,...,610.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,DL,2440,N651DL,SEA,MSP,40,...,557.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,NK,520,N525NK,LAS,MCI,55,...,543.0,6.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,NK,597,N528NK,MSP,FLL,115,...,607.0,25.0,0,0,None,25.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,NK,168,N629NK,PHX,ORD,125,...,632.0,43.0,0,0,None,43.0,0.0,0.0,0.0,0.0
5,2015,1,1,4,DL,95,N320US,SLC,ATL,140,...,725.0,10.0,0,0,None,NaN,NaN,NaN,NaN,NaN
6,2015,1,1,4,HA,17,N389HA,LAS,HNL,145,...,610.0,15.0,0,0,None,0.0,0.0,15.0,0.0,0.0
7,2015,1,1,4,B6,304,N607JB,SJU,JFK,155,...,501.0,11.0,0,0,None,NaN,NaN,NaN,NaN,NaN
8,2015,1,1,4,UA,1162,N37293,BQN,EWR,259,...,605.0,6.0,0,0,None,NaN,NaN,NaN,NaN,NaN
9,2015,1,1,4,B6,1030,N239JB,BQN,MCO,307,...,520.0,20.0,0,0,None,20.0,0.0,0.0,0.0,0.0


In [25]:
transformed_ds.count()

236561

In [ ]:
# Explicitly stop
ray.shutdown()
assert not ray.is_initialized()

## Ejemplo: Ingesta de ML a gran escala

En este ejemplo, aprenderá a crear, desplegar y escalar una canalización de ingestión aleatoria de aprendizaje automático utilizando Ray Dataset y Dataset Pipelines.

* Cómo construir una tubería de ingesta aleatoria que carga, mezcla y alimenta datos en entrenadores distribuidos en unas pocas líneas de código;

* Cómo escalar el pipeline desde la ingesta de datos de 100MiB a datos de 500GiB.

![Image](https://docs.ray.io/en/master/_images/dataset-repeat-2.svg)


In [ ]:
import pyarrow
from ray.data.dataset_pipeline import DatasetPipeline
from ray.data.datasource.datasource import RandomIntRowDatasource
import tempfile
from typing import List

### Construir una cadena de ingesta aleatoria

Un proceso de ingesta típico de aprendizaje automático consta de los 4 pasos siguientes:

* Cargar los datos de entrenamiento desde un almacenamiento externo;
* Iterar sobre los datos durante varias épocas;
* En cada epoch, aplicar global shuffle para decorrelacionar los datos;
* En cada época, dividir los datos barajados en fragmentos, y alimentar los fragmentos a los entrenadores distribuidos;

Veamos cómo implementamos este proceso utilizando el conjunto de datos Ray:

In [ ]:
def create_shuffle_pipeline(
    training_data_dir: str, num_epochs: int, num_shards: int
) -> List[DatasetPipeline]:

    return (
        ray.data.read_parquet(training_data_dir)
        .repeat(num_epochs)
        .random_shuffle_each_window()
        .split(num_shards, equal=True)
    )

Ahora hemos definido una función create_shuffle_pipeline que crea un pipeline de ingestión. Lee training_data_dir, itera durante num_epochs veces, donde en cada epoch baraja y divide los datos de entrenamiento en num_shards.

### Alimentar el pipeline a los entrenadores

Implementemos también un TrainingWorker que consuma los datos barajados de cada shard.

Para simplificar, definiremos un Ray Actor que emule a los training workers. Específicamente,

1. Toma un shard del shuffle pipeline para entrenar;
2. itera sobre el fragmento para obtener un conjunto de datos de entrenamiento por época;
3. Luego consume el conjunto de datos por lotes;

In [ ]:
@ray.remote
class TrainingWorker:
    def __init__(self, rank: int, shard: DatasetPipeline):
        self.rank = rank
        self.shard = shard

    def train(self):
        for epoch, training_dataset in enumerate(self.shard.iter_epochs()):
            # Following code emulates epoch based SGD training.
            print(f"Training... worker: {self.rank}, epoch: {epoch}")
            for i, batch in enumerate(training_dataset.iter_batches()):
                # TODO: replace the code for real training.
                pass

Ejecutémoslo! Ahora vamos a ejecutar el canal de datos de extremo a extremo, primero, analicemos algunos argumentos.

In [ ]:
NUM_TRAINING_WORKERS = 4
NUM_EPOCHS = 5
NUM_COLUMNS = 10
SIZE_100MiB = 100 * 1024 * 1024

# create a local ray cluster.
ray.init()

In [ ]:
print(f"Count of nodes: {len(ray.nodes())}")

In [ ]:
splits = create_shuffle_pipeline(
    filter_files, NUM_EPOCHS, NUM_TRAINING_WORKERS
)

training_workers = [
    TrainingWorker.remote(rank, shard) for rank, shard in enumerate(splits)
]

# Let's run the e2e pipeline
start = time.time()
ray.get([worker.train.remote() for worker in training_workers])
print(f"total ingestion time: {int(time.time() - start)}s")

In [15]:
# To clean the remote process
@ray.remote
class TrainingWorker:
    pass

In [18]:
import threading

@ray.remote
class Task:

    def __init__(self):
        self._thread = threading.Thread(target=self._run, daemon=True)
        self._thread.start()
    
    def _run(self):
        print('Started long-running task')
        time.sleep(10)
        print('You shouldn\'t see this because the process has been killed')

task = Task.remote()
task.__ray_terminate__.remote()   # kills the process with an `exit(0)`

ObjectRef(bbb11aeaabc120eafedc12baffc2e90b28b0138f0500000001000000)

(Task pid=426753) Started long-running task
